# Teorema de Bayes

$
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}
$

Podemos utilizar el Teorema de Bayes para calcular probabilidades condicionales. Cuando uno tiene un dataset completo, realmente no se necesita. El lado izquierdo de la ecuación es fácilmente calculable, y el lado derecho lo es también.

Pero a menudo no tenemos un dataset completo, y en ese caso el Teorema de Bayes es más útil. En este notebook, lo usaremos para resolver varios problemas más desafiantes relacionados con la probabilidad condicional.

## El problema de las galletas

Supón que cuentas con dos bowls llenos de galletas:


*   El bowl 1 cuenta con 30 galletas de vainilla y 10 de chocolate.
*   El bowl 2 cuenta con 20 galletas de vainilla y 20 de chocolate.



Supón ahora que eliges un bowl al azar, y extraes una galleta. Si la galleta extraída fue de vainilla, ¿cuál es la probabilidad de que hayamos elegido el bowl 1?

En otras palabras, estamos buscando $P(B_1|V)$

Lo que nos da el enunciado del problema es:


*   $P(B_1)$ y $ P(B_2)$
*   $P(V|B_1)$
*   $P(V)$, obtenido usando la ley de probabilidad total: $P(V) = P(B_1)P(V|B_1) + P(B_2)P(V|B_2)$



Así pues, podemos obtener que $P(B_1|V)$ = 3/5

Este ejemplo demuestra un uso del teorema de Bayes: proporciona una manera de pasar de
$P(B∣A)$ a
$P(A∣B)$.

## Bayes Diacrónico

Notas...

## Tablas de Bayes

Una herramienta conveniente para hacer una actualización bayesiana es una tabla de Bayes. Uno puede escribir una tabla de Bayes en papel o usar una hoja de cálculo, pero en esta sección usaremos DataFrames de Pandas.

Primero, crearemos un DataFrame vacío con una fila para cada hipótesis:

In [1]:
import pandas as pd

table = pd.DataFrame(index=['Bowl 1', 'Bowl 2'])

Ahora, agregaremos una columna para representar las probabilidades a priori:

In [2]:
table['prior'] = 1/2, 1/2
table

,prior
Bowl 1,0.5
Bowl 2,0.5


Y una columna para los likelihoods o verosimilitudes:

In [3]:
table['likelihood'] = 3/4, 1/2
table

,prior,likelihood
Bowl 1,0.5,0.75
Bowl 2,0.5,0.50


Aquí vemos una diferencia con el método anterior, calculamos verosimilitudes para ambas hipótesis, no solo para el Bowl 1:

* La probabilidad de obtener una galleta de vainilla del Bowl 1 es 3/4.

* La probabilidad de obtener una galleta de vainilla del Bowl 2 es 1/2.

Notar que las verosimilitudes no suman 1. Esto está bien; cada una de ellas es una probabilidad condicionada a una hipótesis diferente. No hay razón para que sumen 1 y no hay problema si no lo hacen.

El siguiente paso es similar a lo que hicimos con el Teorema de Bayes; multiplicamos las probabilidades a priori por las verosimilitudes:







In [4]:
table['unnorm'] = table['prior'] * table['likelihood']
table

,prior,likelihood,unnorm
Bowl 1,0.5,0.75,0.375
Bowl 2,0.5,0.50,0.250


Llamamos al resultado "unnorm" porque estos valores son los "posteriores no normalizados". Cada uno de ellos es el producto de una probabilidad a priori y una verosimilitud:

$P(A∣B)∝P(B∣A)⋅P(A)$

lo cual es el numerador del Teorema de Bayes.

Si los sumamos, tenemos:

$P(B)=∑P(B∣A_i)⋅P(A_i) $

lo cual es el denominador del Teorema de Bayes, $P(B)$.







Así que podemos calcular la probabilidad total de los datos de esta manera:

In [7]:
prob_data = table['unnorm'].sum()
prob_data # 5/8

0.625

Y podemos calcular las probabilidades posteriores de esta manera:

In [8]:
table['posterior'] = table['unnorm'] / prob_data
table

,prior,likelihood,unnorm,posterior
Bowl 1,0.5,0.75,0.375,0.6
Bowl 2,0.5,0.50,0.250,0.4


La probabilidad posterior para el Bowl 1 es 0.6, que es lo que obtuvimos usando explícitamente el Teorema de Bayes. Como beneficio adicional, también obtenemos la probabilidad posterior del Bowl 2, que es 0.4.

Cuando sumamos los posteriores no normalizados y dividimos, forzamos a que los posteriores sumen 1. Este proceso se llama "normalización", por lo que la probabilidad total de los datos también se llama la "constante de normalización".

## El problema de los dados

Supongamos que tengo una caja con un dado de 6 caras, un dado de 8 caras y un dado de 12 caras. Elijo uno de los dados al azar, lo lanzo y reporto que el resultado es un 1. ¿Cuál es la probabilidad de que haya elegido el dado de 6 caras?

In [9]:
table2 = pd.DataFrame(index=[6, 8, 12])

Creamos nuestra tabla:

In [10]:
from fractions import Fraction

table2['prior'] = Fraction(1, 3)
table2['likelihood'] = Fraction(1, 6), Fraction(1, 8), Fraction(1, 12)
table2

,prior,likelihood
6,1/3,1/6
8,1/3,1/8
12,1/3,1/12


Y definimos una función que agrupa nuestros pasos anteriores:

In [11]:
def update(table):
    """Compute the posterior probabilities."""
    table['unnorm'] = table['prior'] * table['likelihood']
    prob_data = table['unnorm'].sum()
    table['posterior'] = table['unnorm'] / prob_data
    return prob_data

In [12]:
prob_data = update(table2)

In [13]:
table2

,prior,likelihood,unnorm,posterior
6,1/3,1/6,1/18,4/9
8,1/3,1/8,1/24,1/3
12,1/3,1/12,1/36,2/9


La probabilidad posterior del dado de 6 caras es 4/9, lo cual es un poco más que las probabilidades para los otros dados, que son 3/9 y 2/9. Intuitivamente, el dado de 6 caras es el más probable porque tenía la mayor verosimilitud de producir el resultado que observamos.

## El problema de Monty Hall

A continuación, usaremos una tabla de Bayes para resolver uno de los problemas más controvertidos en probabilidad.

El problema de Monty Hall se basa en un programa de televisión llamado "Let's Make a Deal". Si eres un concursante en el programa, así es cómo funciona el juego:

El presentador, Monty Hall, te muestra tres puertas cerradas, numeradas como 1, 2 y 3, y te dice que hay un premio detrás de cada puerta.

Uno de los premios es valioso (tradicionalmente un coche), y los otros dos son menos valiosos (tradicionalmente cabras).

El objetivo del juego es adivinar detrás de qué puerta está el coche. Si aciertas, te llevas el coche.

Supongamos que eliges la Puerta 1. Antes de abrir la puerta que elegiste, Monty abre la Puerta 3 y revela una cabra. Luego, Monty te ofrece la opción de quedarte con tu elección original o cambiar a la otra puerta que no ha sido abierta.

Para maximizar tus posibilidades de ganar el coche, ¿deberías quedarte con la Puerta 1 o cambiar a la Puerta 2?


Para responder a esta pregunta, tenemos que hacer algunas suposiciones sobre el comportamiento del presentador:

- Monty siempre abre una puerta y te ofrece la opción de cambiar.
- Nunca abre la puerta que elegiste originalmente ni la puerta con el coche.
- Si eliges la puerta con el coche, elige una de las otras puertas al azar.

Bajo estas suposiciones, es mejor cambiar. Si te quedas, ganas $\frac{1}{3}$ del tiempo. Si cambias, ganas $\frac{2}{3}$ del tiempo.

Si no has encontrado este problema antes, podrías encontrar sorprendente esa respuesta. No estarías solo; muchas personas tienen la fuerte intuición de que no importa si te quedas o cambias. Hay dos puertas restantes, razonan, así que la probabilidad de que el coche esté detrás de la Puerta 1 es del 50%. Pero eso es incorrecto.

Para entender por qué, puede ser útil usar una tabla de Bayes. Comenzamos con tres hipótesis: el coche podría estar detrás de la Puerta 1, 2 o 3. Según el enunciado del problema, la probabilidad a priori para cada puerta es $\frac{1}{3} $.

In [14]:
table3 = pd.DataFrame(index=['Door 1', 'Door 2', 'Door 3'])
table3['prior'] = Fraction(1, 3)
table3

,prior
Door 1,1/3
Door 2,1/3
Door 3,1/3


Los datos indican que Monty abrió la Puerta 3 y reveló una cabra. Así que consideremos la probabilidad de los datos bajo cada hipótesis:

- Si el coche está detrás de la Puerta 1, Monty elige la Puerta 2 o 3 al azar, por lo tanto, la probabilidad de que abra la Puerta 3 es $\frac{1}{2} $.

- Si el coche está detrás de la Puerta 2, Monty debe abrir la Puerta 3, por lo tanto, la probabilidad de los datos bajo esta hipótesis es 1.

- Si el coche está detrás de la Puerta 3, Monty no la abre, por lo tanto, la probabilidad de los datos bajo esta hipótesis es 0.

Aquí están las verosimilitudes.


In [15]:
table3['likelihood'] = Fraction(1, 2), 1, 0
table3

,prior,likelihood
Door 1,1/3,1/2
Door 2,1/3,1
Door 3,1/3,0


Ahora que tenemos las probabilidades a priori y las verosimilitudes, podemos usar la actualización para calcular las probabilidades posteriores.


In [16]:
update(table3)
table3

,prior,likelihood,unnorm,posterior
Door 1,1/3,1/2,1/6,1/3
Door 2,1/3,1,1/3,2/3
Door 3,1/3,0,0,0


Después de que Monty abre la Puerta 3, la probabilidad posterior de la Puerta 1 es $1/3$, y la probabilidad posterior de la Puerta 2 es $2/3$. Por lo tanto, es mejor cambiar de la Puerta 1 a la Puerta 2.

Como este ejemplo muestra, nuestra intuición sobre la probabilidad no siempre es confiable. El Teorema de Bayes puede ayudar proporcionando una estrategia de divide y vencerás:

- Primero, escribe las hipótesis y los datos.

- Luego, determina las probabilidades a priori.

- Finalmente, calcula la verosimilitud de los datos bajo cada hipótesis.

La tabla de Bayes hace el resto del trabajo.


## Ejercicios

Ahora, supongamos que Monty siempre elige la Puerta 2 si puede, y solo elige la Puerta 3 si tiene que hacerlo (porque el coche está detrás de la Puerta 2).

- Si eliges la Puerta 1 y Monty abre la Puerta 2, ¿cuál es la probabilidad de que el coche esté detrás de la Puerta 3?

- Si eliges la Puerta 1 y Monty abre la Puerta 3, ¿cuál es la probabilidad de que el coche esté detrás de la Puerta 2?

### Referencias

https://allendowney.github.io/ThinkBayes2/chap02.html#the-cookie-problem